# 🔄 RDD Introduction: The Foundation of Spark

**Time to complete:** 25 minutes  
**Difficulty:** Beginner  
**Prerequisites:** Spark Session basics

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand what RDDs (Resilient Distributed Datasets) are
- ✅ Learn how to create RDDs from different sources
- ✅ Know the difference between transformations and actions
- ✅ Understand RDD persistence and fault tolerance
- ✅ Be able to perform basic RDD operations

---

## 🔍 What is an RDD?

**RDD** stands for **Resilient Distributed Dataset**. It's Spark's fundamental data structure.

### Key Characteristics:
- **Resilient**: Fault-tolerant, automatically recovers from failures
- **Distributed**: Data is split across multiple machines
- **Dataset**: Collection of data that can be processed in parallel

### Why RDDs Matter:
- Foundation of all Spark operations
- Enable distributed processing
- Provide fault tolerance
- Support complex transformations

**Think of RDDs as distributed, fault-tolerant collections that can be processed in parallel.**

## 🏗️ Creating RDDs

There are several ways to create RDDs in PySpark:

In [ ]:
# First, create a Spark Session (if not already created)
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RDD_Introduction") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

print(f"✅ Spark Session ready - Version: {spark.version}")

### Method 1: From a Python Collection

In [ ]:
# Create RDD from a Python list
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rdd_from_list = sc.parallelize(data)

print("✅ RDD created from Python list")
print(f"Type: {type(rdd_from_list)}")
print(f"Number of partitions: {rdd_from_list.getNumPartitions()}")
print(f"First element: {rdd_from_list.first()}")
print(f"Total elements: {rdd_from_list.count()}")

### Method 2: From a Text File

In [ ]:
# Create a sample text file for demonstration
sample_text = """Hello World
Welcome to PySpark
RDDs are powerful
Distributed computing
Fault tolerance"""

with open("sample_text.txt", "w") as f:
    f.write(sample_text)

# Create RDD from text file
text_rdd = sc.textFile("sample_text.txt")

print("✅ RDD created from text file")
print(f"Lines in file: {text_rdd.count()}")
print("Content:")
for line in text_rdd.collect():
    print(f"  '{line}'")

### Method 3: From Key-Value Pairs

In [ ]:
# Create RDD with key-value pairs
kv_data = [("Alice", 25), ("Bob", 30), ("Charlie", 35), ("Diana", 28)]
kv_rdd = sc.parallelize(kv_data)

print("✅ RDD created with key-value pairs")
print("Content:")
for item in kv_rdd.collect():
    print(f"  {item[0]}: {item[1]}")

# Check if it's a pair RDD
print(f"\nIs Pair RDD: {kv_rdd.map(lambda x: isinstance(x, tuple) and len(x) == 2).reduce(lambda a, b: a and b)}")

## 🔄 Transformations vs Actions

This is the most important concept in Spark!

### Transformations (Lazy Operations)

**Transformations create new RDDs from existing ones.**

They are **lazy** - they don't execute immediately. Spark builds a DAG (Directed Acyclic Graph) of transformations to optimize execution.

In [ ]:
# Create a base RDD
numbers = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# Transformations (lazy - create execution plan)
doubled = numbers.map(lambda x: x * 2)        # Double each number
filtered = doubled.filter(lambda x: x > 5)    # Keep numbers > 5
squared = filtered.map(lambda x: x * x)       # Square the results

print("✅ Transformations defined (but not executed yet)")
print(f"Original RDD: {numbers.collect()}")
print("Transformations created a DAG, but no computation happened yet!")

### Actions (Eager Operations)

**Actions trigger actual computation and return results.**

They execute the DAG of transformations and bring results back to the driver.

In [ ]:
# Actions (eager - trigger computation)
print("🎯 Executing transformations with actions:")

# Action 1: collect() - brings all data to driver
result = squared.collect()
print(f"Final result: {result}")

# Action 2: count() - counts elements
total_count = squared.count()
print(f"Total elements: {total_count}")

# Action 3: first() - gets first element
first_element = squared.first()
print(f"First element: {first_element}")

# Action 4: take() - gets first N elements
first_five = squared.take(5)
print(f"First 5 elements: {first_five}")

### 🔄 The Lazy Evaluation Process

```
Transformations (Lazy) → Build DAG → Action (Eager) → Execute DAG → Return Results
```

**Why lazy evaluation matters:**
- Allows Spark to optimize the execution plan
- Reduces unnecessary computations
- Enables fault tolerance
- Supports distributed processing

## 🛠️ Common RDD Operations

Let's explore some essential RDD operations:

In [ ]:
# Create sample data for demonstrations
sample_data = ["hello world", "welcome to pyspark", "rdd operations", "distributed computing"]
sample_rdd = sc.parallelize(sample_data)

print("📊 Sample RDD Operations:")
print(f"Original: {sample_rdd.collect()}")

# map() - Transform each element
upper_rdd = sample_rdd.map(lambda x: x.upper())
print(f"\nmap() - Uppercase: {upper_rdd.collect()}")

# flatMap() - Transform and flatten
words_rdd = sample_rdd.flatMap(lambda x: x.split())
print(f"\nflatMap() - Split into words: {words_rdd.collect()}")

# filter() - Keep elements matching condition
long_words = words_rdd.filter(lambda x: len(x) > 4)
print(f"\nfilter() - Words longer than 4 chars: {long_words.collect()}")

# distinct() - Remove duplicates
distinct_words = words_rdd.distinct()
print(f"\ndistinct() - Unique words: {distinct_words.collect()}")

## 🧮 Numeric RDD Operations

In [ ]:
# Create numeric RDD
nums = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

print("🔢 Numeric RDD Operations:")
print(f"Numbers: {nums.collect()}")

# Basic aggregations
print(f"Sum: {nums.sum()}")
print(f"Mean: {nums.mean():.2f}")
print(f"Max: {nums.max()}")
print(f"Min: {nums.min()}")

# Custom reduction
product = nums.reduce(lambda x, y: x * y)
print(f"Product: {product}")

# Fold (with initial value)
sum_with_start = nums.fold(100, lambda x, y: x + y)
print(f"Sum with initial 100: {sum_with_start}")

## 💾 RDD Persistence

RDDs can be cached in memory for faster access:

In [ ]:
# Create a large RDD for caching demonstration
large_data = list(range(1, 1001))  # 1 to 1000
large_rdd = sc.parallelize(large_data)

# Apply some transformations
processed_rdd = large_rdd \
    .map(lambda x: x * 2) \
    .filter(lambda x: x % 3 == 0) \
    .map(lambda x: x + 10)

print("📈 Large RDD processing:")

# First execution (no cache)
import time
start_time = time.time()
count1 = processed_rdd.count()
time1 = time.time() - start_time

print(f"First execution: {count1} elements in {time1:.3f} seconds")

# Cache the RDD
processed_rdd.cache()
print("\n💾 RDD cached in memory")

# Second execution (with cache)
start_time = time.time()
count2 = processed_rdd.count()
time2 = time.time() - start_time

print(f"Second execution: {count2} elements in {time2:.3f} seconds")
print(f"Speedup: {time1/time2:.2f}x faster!")

# Check storage level
print(f"Storage level: {processed_rdd.getStorageLevel()}")

## 🛡️ Fault Tolerance

RDDs automatically recover from failures through lineage:

In [ ]:
# RDD Lineage (execution plan)
test_rdd = sc.parallelize([1, 2, 3, 4, 5]) \
    .map(lambda x: x * 2) \
    .filter(lambda x: x > 4) \
    .map(lambda x: x + 10)

print("🛡️ RDD Fault Tolerance:")
print("Lineage (execution plan):")
for line in test_rdd.toDebugString().split('\n'):
    print(f"  {line}")

print(f"\nResult: {test_rdd.collect()}")
print("\nIf any partition fails, Spark can recreate it using this lineage!")

## 🧹 Cleaning Up

In [ ]:
# Clean up sample files
import os
if os.path.exists("sample_text.txt"):
    os.remove("sample_text.txt")

print("🧹 Cleanup completed")

# Note: Spark Session will be cleaned up automatically in Jupyter
# In production code, use: spark.stop()

## 🎯 Key Takeaways

### What You Learned:
- ✅ **RDDs** are Spark's fundamental distributed data structure
- ✅ **Transformations** are lazy operations that build execution plans
- ✅ **Actions** trigger actual computation and return results
- ✅ **Caching** improves performance for reused RDDs
- ✅ **Lineage** provides fault tolerance

### RDD Characteristics:
- 🔸 **Immutable**: Cannot be changed after creation
- 🔸 **Distributed**: Partitioned across cluster nodes
- 🔸 **Fault-tolerant**: Automatically recovered via lineage
- 🔸 **Lazy evaluation**: Transformations don't execute immediately

### Common Operations:
- 🔸 `map()` - Transform each element
- 🔸 `filter()` - Keep elements matching condition
- 🔸 `flatMap()` - Transform and flatten results
- 🔸 `collect()` - Bring results to driver
- 🔸 `count()` - Count elements

---

## 🚀 Next Steps

Now that you understand RDDs, you're ready for:

1. **DataFrame Introduction** - Higher-level structured data processing
2. **Transformations vs Actions Deep Dive** - Understanding lazy evaluation
3. **RDD Mastery** - Advanced RDD operations and performance

---

**🎉 Congratulations! You now understand the foundation of Spark - Resilient Distributed Datasets!**